In [1]:
from IPython.display import display
import IPython
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import sklearn as skl
%matplotlib notebook
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

# Regret-based Algorithm

In [5]:
n = int(input(print('Enter the number of rows in a matrix')))
print(n)

Enter the number of rows in a matrix
10
10


In [67]:
np.random.seed(33)
mat=pd.DataFrame(np.random.randint(0,high=50,size=(n,n),dtype=int))
print(mat)

    0   1   2   3   4   5   6   7   8   9
0  20   7  24   2  18   9   3  38  19  14
1  42  45  35  33  13  31  44  39  12  11
2  22  16  16  42  38  16  20  24  40  19
3  23   9  35  19   7  10   3   7  18  17
4   3  43  43  22  26  32  48  44  25  14
5  30  26  11   1  49  15  36  33  11  33
6  47  15  14  14  28  40  10  24  44  28
7  21  42   8   0  25  13  23  47  46  39
8  25  22   9  49  43   5  46  10  27  32
9  19  43  32  16  41  48  28  10   4   8


In [68]:
#assign initial job list
row_job = np.arange(n) + 1 #+1 for real job number
col_job = np.arange(n) + 1 #+1 for real job number
#substract row_min from mat
pat = np.array(mat)

print(pat)

[[20  7 24  2 18  9  3 38 19 14]
 [42 45 35 33 13 31 44 39 12 11]
 [22 16 16 42 38 16 20 24 40 19]
 [23  9 35 19  7 10  3  7 18 17]
 [ 3 43 43 22 26 32 48 44 25 14]
 [30 26 11  1 49 15 36 33 11 33]
 [47 15 14 14 28 40 10 24 44 28]
 [21 42  8  0 25 13 23 47 46 39]
 [25 22  9 49 43  5 46 10 27 32]
 [19 43 32 16 41 48 28 10  4  8]]


In [69]:
# 스택을 이용하여 job index를 행, 열 번호와 상관없이 유지한다.
stack = np.zeros((n*2))
def stack_push_jobs(top, element):
  if top == -1:#empty stack
    stack[top+1] = element #[rowjob,coljob]
    return top + 1

  if stack[top] != element: # 현재 top # 이미 있는 거랑 중복이 아니면
    stack[top+1] = element
    return top + 1
  else:
    return top

In [70]:
#step 0 : Cmax를 비롯한 변수 초기화 & step 반복문 생성
new_n = n
Cmax = 0
top = -1
# for al_iter in range(n):

## step by step

In [30]:
#<step 1> 각 행과 열의 최솟값 row_min과 col_min을 구해서 Cmax와 행렬 pat을 갱신
row_min = np.amin(pat, axis = 1)
for i in range(new_n):
  pat[i] = pat[i] - row_min[i]
#substract col_min from mat
col_min = np.amin(pat, axis = 0)
pat = pat - col_min
#calculate value of Cmax
for i in range(new_n):
  Cmax += row_min[i] + col_min[i]
  print(f"Cmax{i+1}: {Cmax}")



Cmax1: 2
Cmax2: 13
Cmax3: 29
Cmax4: 32
Cmax5: 37
Cmax6: 38
Cmax7: 48
Cmax8: 52
Cmax9: 57
Cmax10: 61


In [32]:
regret_mat = np.zeros((new_n,new_n))
row_regret_min = 0
col_regret_min = 0
for i in range(new_n):
  for j in range(new_n):
    if(pat[i,j] == 0):
      row_regret_min = 30000000
      col_regret_min = 30000000
      for k in range(new_n): #i빼고 col min 구하기
        if((k != i) & (pat[k,j] < col_regret_min)):
          col_regret_min = pat[k,j]
          #print(col_regret_min)
        if((k != j) & (pat[i,k] < row_regret_min)):
          row_regret_min = pat[i,k]
          #print(row_regret_min)
        regret_mat[i,j] = row_regret_min + col_regret_min
print(regret_mat)

[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  0.  0.  0.  0.  3.]
 [ 0.  5.  4.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [17.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 10.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  4.  0.  0.  0.]
 [ 0.  0.  0.  8.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  3.  0.]]


In [35]:
idx = np.unravel_index(np.argmax(regret_mat, axis=None), regret_mat.shape)
print(idx,regret_mat[idx])

(4, 0) 17.0


In [36]:
# <step 4-1> job을 배치하고 배치한 job들의 반대 순서쌍에 infinite 부여
assigned_row_job = row_job[idx[0]]
assigned_col_job = col_job[idx[1]]
print("assigned_row_job: ", assigned_row_job)
print("assigned_col_job: ", assigned_col_job)
#stack push
top = stack_push_jobs(top, assigned_row_job)
top = stack_push_jobs(top, assigned_col_job)
idx1 = np.where(row_job == assigned_col_job) #top
idx2 = np.where(col_job == assigned_row_job) #top -1
print("idx1: ", idx1)
print("idx2: ", idx2)





assigned_row_job:  5
assigned_col_job:  1
idx1:  (array([0]),)
idx2:  (array([4]),)


In [38]:
# 순서쌍이 서로 연결되지 않을 경우 예외처리
tmp1 = np.array(idx1)
tmp2 = np.array(idx2)
print(tmp1.shape)
print(tmp2.shape)
if tmp1.shape == (1,0):
  idx1 = np.where(row_job == stack[top-1])
  idx2 = np.where(col_job == stack[top-2])

if tmp2.shape == (1,0):
  idx2 = np.where(col_job == stack[top-2])

print("idx1: ", idx1)
print("idx2: ", idx2)
pat[idx1,idx2] = 30000000
print(row_job)
print(col_job)

(1, 1)
(1, 1)
idx1:  (array([0]),)
idx2:  (array([4]),)
[[      18        5       22        0 30000000        7        1       32
        17       12]
 [      31       34       24       22        0       20       33       24
         1        0]
 [       6        0        0       26       20        0        4        4
        24        3]
 [      20        6       32       16        2        7        0        0
        15       14]
 [       0       40       40       19       21       29       45       37
        22       11]
 [      29       25       10        0       46       14       35       28
        10       32]
 [      37        5        4        4       16       30        0       10
        34       18]
 [      21       42        8        0       23       13       23       43
        46       39]
 [      20       17        4       44       36        0       41        1
        22       27]
 [      15       39       28       12       35       44       24        2
         0     

In [41]:
#<step 4-2> 해당 index를 job list에서 삭제하고 setup time 행렬에서도 삭제
row_job = np.delete(row_job, idx[0])
col_job = np.delete(col_job, idx[1])

pat = np.delete(pat, idx[0], 0)
pat = np.delete(pat, idx[1], 1)
pat #reduced matrix
print(pat)
new_n = new_n - 1
#이 후 다시 step 1로 돌아와서 반복

[ 1  2  3  4  6  7  8  9 10]
[ 2  3  4  5  6  7  8  9 10]
[[       5       22        0 30000000        7        1       32       17
        12]
 [      34       24       22        0       20       33       24        1
         0]
 [       0        0       26       20        0        4        4       24
         3]
 [       6       32       16        2        7        0        0       15
        14]
 [      25       10        0       46       14       35       28       10
        32]
 [       5        4        4       16       30        0       10       34
        18]
 [      42        8        0       23       13       23       43       46
        39]
 [      17        4       44       36        0       41        1       22
        27]
 [      39       28       12       35       44       24        2        0
         4]]


## All at once

In [72]:
for al_iter in range(n):
  print(f'[Iter {al_iter+1}]')
  row_min = np.amin(pat, axis = 1)
  for i in range(new_n):
    pat[i] = pat[i] - row_min[i]
    #substract col_min from mat
  col_min = np.amin(pat, axis = 0)
  pat = pat - col_min
    #calculate value of Cmax
  for i in range(new_n):
    Cmax += row_min[i] + col_min[i]
  print(f"1. Cmax: {Cmax}")
  regret_mat = np.zeros((new_n,new_n))
  row_regret_min = 0
  col_regret_min = 0
  for i in range(new_n):
    for j in range(new_n):
      if(pat[i,j] == 0):
        row_regret_min = 30000000
        col_regret_min = 30000000
        for k in range(new_n): #i빼고 col min 구하기
          if((k != i) & (pat[k,j] < col_regret_min)):
            col_regret_min = pat[k,j]
          #print(col_regret_min)
          if((k != j) & (pat[i,k] < row_regret_min)):
            row_regret_min = pat[i,k]
          #print(row_regret_min)
        regret_mat[i,j] = row_regret_min + col_regret_min
        #print(regret_mat)
  idx = np.unravel_index(np.argmax(regret_mat, axis=None), regret_mat.shape)
  #print(f'{idx}')
  assigned_row_job = row_job[idx[0]]
  assigned_col_job = col_job[idx[1]]
  print('2.Assigned Job Index')
  print("assigned_row_job: ", assigned_row_job)
  print("assigned_col_job: ", assigned_col_job)
  #stack push
  top = stack_push_jobs(top, assigned_row_job)
  top = stack_push_jobs(top, assigned_col_job)
  idx1 = np.where(row_job == assigned_col_job) #top
  idx2 = np.where(col_job == assigned_row_job) #top -1
  print('3. Stack 내 top index')
  print("idx1: ", idx1)
  print("idx2: ", idx2)

  # 순서쌍이 서로 연결되지 않을 경우 예외처리
  tmp1 = np.array(idx1)
  tmp2 = np.array(idx2)
  print(tmp1.shape)
  print(tmp2.shape)
  if tmp1.shape == (1,0):
    print('#예외처리')
    idx1 = np.where(row_job == stack[top-1])
    idx2 = np.where(col_job == stack[top-2])
    print("idx1: ", idx1)

  if tmp2.shape == (1,0):
    idx2 = np.where(col_job == stack[top-2])
    print("idx2: ", idx2)

  
  
  pat[idx1,idx2] = 30000000
  print('4. Assigned job 제거 확인')
  print(f'Current_row_job : {row_job}')
  print(f'Current_col_job : {col_job}')
  row_job = np.delete(row_job, idx[0])
  col_job = np.delete(col_job, idx[1])
  print(f'Deleted_row_job : {row_job}')
  print(f'Deleted_col_job : {col_job}')

  print('5. Assigned job 제거 후 setup time 행렬(pat)')
  pat = np.delete(pat, idx[0], 0)
  pat = np.delete(pat, idx[1], 1)
  print(pd.DataFrame(pat,index=row_job,columns=col_job))

  print('----------------------------------------------------------------------------------')
  new_n = new_n - 1

[Iter 1]
1. Cmax: 61
2.Assigned Job Index
assigned_row_job:  5
assigned_col_job:  1
3. Stack 내 top index
idx1:  (array([0]),)
idx2:  (array([4]),)
(1, 1)
(1, 1)
4. Assigned job 제거 확인
Current_row_job : [ 1  2  3  4  5  6  7  8  9 10]
Current_col_job : [ 1  2  3  4  5  6  7  8  9 10]
Deleted_row_job : [ 1  2  3  4  6  7  8  9 10]
Deleted_col_job : [ 2  3  4  5  6  7  8  9 10]
5. Assigned job 제거 후 setup time 행렬(pat)
    2   3   4         5   6   7   8   9   10
1    5  22   0  30000000   7   1  32  17  12
2   34  24  22         0  20  33  24   1   0
3    0   0  26        20   0   4   4  24   3
4    6  32  16         2   7   0   0  15  14
6   25  10   0        46  14  35  28  10  32
7    5   4   4        16  30   0  10  34  18
8   42   8   0        23  13  23  43  46  39
9   17   4  44        36   0  41   1  22  27
10  39  28  12        35  44  24   2   0   4
----------------------------------------------------------------------------------
[Iter 2]
1. Cmax: 61
2.Assigned Job Index
assigned